In [61]:
import datetime
import hnswlib
import random
import numpy as np
import copy
import matplotlib.pyplot as plt
import hnswlib
import pandas as pd
import matplotlib.pyplot as plt
import time
import numpy as np
from sklearn import datasets
class pure_dbscan(object):
    def __init__(self,data,eps,minpts):
        self.data = data
        self.eps = eps
        self.minpts=minpts
        
    def findNeighbor(self,j):
        innerMC=[]
        neighbor = []
        for i in range(self.data.shape[0]):
            temp = np.sqrt(np.sum(np.square(self.data[j] - self.data[i])))  # 计算欧式距离
            if temp <= self.eps:
                neighbor.append(i)
            if temp <= 0.5*self.eps:
                innerMC.append(i)
        return list(set(neighbor)),list(set(innerMC))
    def asignLable(self):                  #使用HNSW查找每个数据点的最近邻
        emptyPoinnt=[]
        data_label = list(range(self.data.shape[0])) #根据数据集的长度对每个数据点进行标记。第一个数据点，label为0。
   
        core = []
        neighbor_dict = {}
        noise=[]

        while len(data_label) != 0:
            center = data_label[0]
            data_label.remove(center)  # 把查询点删除了。
            neighbor,innerMC = self.findNeighbor(center)  #查找它的领域半径邻居，和内环邻居。
            if len(neighbor) >=self.minpts:
                core.append(center)

                # 在内环的邻居数大于密度阈值时，才进行查询。
                if len(innerMC)>=self.minpts:
                    core=core+innerMC
                    for i in innerMC:
                        if i not in neighbor_dict.keys():
                            neighbor_dict[i]=[]
                    data_label = list(set(data_label) - set(innerMC))

            if len(neighbor)==1:    #判断此点是否为空值点
                noise.append(center)
            neighbor_dict[center] = neighbor
        core = set(core)
        noise=set(noise)
        return neighbor_dict,core,noise
    def DBSCAN(self):
        k = -1          #初始化聚类簇数 k=-1
        gama = set([x for x in range(len(self.data))])  # 初始化未访问样本集合：gama
        cluster = [-1 for _ in range(len(self.data))]  # 聚类
        neighbor_list,omega_list,noise=self.asignLable()
        
        while len(omega_list) > 0:
            gama_old = copy.deepcopy(gama)
            j = random.choice(list(omega_list))  # 随机选取一个核心对象
            k = k + 1
            Q = list()
            Q.append(j)
            gama.remove(j)
       
            while len(Q) > 0:
                q = Q[0]
                Q.remove(q)
                if len(neighbor_list[q]) >= self.minpts:
                # if q  in list(omega_list):
                    delta = set(neighbor_list[q]) & gama
                    deltalist = list(delta)
                    for i in range(len(delta)):
                        Q.append(deltalist[i])
                        gama = gama - delta
            Ck = gama_old - gama
            Cklist = list(Ck)
            for i in range(len(Ck)):
                cluster[Cklist[i]] = k
            omega_list = omega_list - Ck
        gama=gama-noise
        for i in gama:
            neihbor_noise=neighbor_list[i]
            number = set(neihbor_noise).intersection(omega_list)
            if len(number)==0:
                continue
            if len(number)!=0:
                cluster[i]=cluster[list(number)[0]]
        return cluster
    

iris = datasets.load_iris()
data=iris.data   

student = pure_dbscan(data,0.8,9)

begin = datetime.datetime.now()
cluster=student.DBSCAN()
end = datetime.datetime.now()
print('--------- 我是可爱的分割线-----------')
totalTime = (end - begin).total_seconds()
print (totalTime)
       



--------- 我是可爱的分割线-----------
0.158578
